## Perform functional connectivity analysis on trial data, eyes open, and eyes closed data for all viable CLBP subjects
#### Input: *stc.mat
####  Output: con, visualization

In [1]:
import os
from pathlib import Path

import mne
import numpy as np
import scipy.io as sio
from mne_connectivity import (spectral_connectivity_epochs,
                              spectral_connectivity_time,)
import sys
sys.path.append("/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/")
from eeg_toolkit import preprocess, utils
from eeg_toolkit.functional_connectivity import *

ModuleNotFoundError: No module named 'utils'

In [9]:
# Settings
times_tup, time_win_path = preprocess.get_time_window(5)

data_dir = Path("../../../Data")
EC_resting_data_path = data_dir / "STC_EC_resting"
EO_resting_data_path = data_dir / "STC_EO_resting"
Z_scored_epochs_data_path = data_dir / "STC_Z_scored_Epochs"

EC_resting_save_path = data_dir / "FC_EC_resting"
EO_resting_save_path = data_dir / "FC_EO_resting"
Z_scored_epochs_save_path = data_dir / "FC_Z_scored_Epochs" / time_win_path

save_paths = [EC_resting_save_path, EO_resting_save_path, Z_scored_epochs_save_path]

[os.makedirs(path) for path in save_paths if not os.path.exists(path)]

# Noise cov: crop raw during eyes-open resting condition
noise_cov_win = 5.5, 7.5  # rest_min, rest_max

[-2.5,0.0,2.5]


In [10]:
# viable subjects
sub_ids = utils.import_subs(data_dir, "sub_ids.txt")
sub_ids = sub_ids[7:]  # select only 64ch subs and turn into set
print(sub_ids)

['018', '020', '021', '022', '023', '024', '027', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', 'C1.', 'C2.', 'C3.', 'C5.', 'C6.', 'C7.', 'C8.', 'C9.', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C24', 'C25', 'C26', 'C27']


In [26]:
# Get stc only from selected labels
roi_names = [  # Left
    "rostralanteriorcingulate-lh",  # Left Rostral ACC
    "caudalanteriorcingulate-lh",  # Left Caudal ACC
    "postcentral-lh",  # Left S1,
    "insula-lh",
    "superiorfrontal-lh",  # Left Insula, Left DL-PFC,
    "medialorbitofrontal-lh",  # Left Medial-OFC
    # Right
    "rostralanteriorcingulate-rh",  # Right Rostral ACC
    "caudalanteriorcingulate-rh",  # Right Caudal ACC
    "postcentral-rh",  # , Right S1
    "insula-rh",
    "superiorfrontal-rh",  # Right Insula, Right DL-PFC
    "medialorbitofrontal-rh",
]  # Right Medial-OFC

Freq_Bands = {
    # 'delta': [0, 4],
    "theta": [4.0, 8.0],
    "alpha": [8.0, 13.0],
    "beta": [13.0, 30.0],
    "low-gamma": [30.0, 58.5],
    "notch": [58.5, 61.5],
    "high-gamma": [61.5, 100.0],
}

sfreq = 400  # Hz
tmin, tmax = 0.0, 1.0  # exclude the baseline period for connectivity estimation

In [27]:
for sub_id in sub_ids:
    # Epochs
    sub_data_path_zepo = Z_scored_epochs_data_path / time_win_path / sub_id
    con_methods = [
        "wpli",
        "dpli",
        "plv",
        # "",
    ]
    compute_connectivity_epochs(
        sub_data_path_zepo, roi_names, con_methods, Freq_Bands, sfreq
    )

    # Resting state
    # eyes closed
    condition = "eyes open"  # replace with the actual condition
    sub_data_path_EO = f"{data_path_EO}/{time_win_path}/{sub_id}/"
    # compute_connectivity_resting_state(sub_data_path_EO, roi_names, con_methods, freq_bounds, sfreq, condition)

    # eyes open
    condition = "eyes closed"  # replace with the actual condition
    sub_data_path_EC = f"{data_path_EC}/{time_win_path}/{sub_id}/"
    # compute_connectivity_resting_state(sub_data_path_EC, roi_names, con_methods, freq_bounds, sfreq, condition)

Not setting metadata
73 matching events found
No baseline correction applied
0 projection items activated
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..5.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (20 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: WPLI
    computing cross-spectral density for epochs 1..1


/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/functional_connectivity.py:37: RuntimeWarning: There were no Annotations stored in <EpochsArray |  73 events (all good), 0 – 5 s, baseline off, ~13.4 MB, data loaded,
 '1': 73>, so metadata was not modified.
  con = mne_connectivity.spectral_connectivity_epochs(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


    computing cross-spectral density for epochs 2..2


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


    computing cross-spectral density for epochs 3..3


...

    computing cross-spectral density for epochs 57..57
    computing cross-spectral density for epochs 58..58
    computing cross-spectral density for epochs 59..59
    computing cross-spectral density for epochs 60..60
    computing cross-spectral density for epochs 61..61
    computing cross-spectral density for epochs 62..62
    computing cross-spectral density for epochs 63..63
    computing cross-spectral density for epochs 64..64
    computing cross-spectral density for epochs 65..65


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


    computing cross-spectral density for epochs 66..66
    computing cross-spectral density for epochs 67..67
    computing cross-spectral density for epochs 68..68
    computing cross-spectral density for epochs 69..69
    computing cross-spectral density for epochs 70..70
    computing cross-spectral density for epochs 71..71
    computing cross-spectral density for epochs 72..72
    computing cross-spectral density for epochs 73..73
    assembling connectivity matrix
[Connectivity computation done]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


NameError: name 'con_epochs' is not defined

## testing

In [ ]:
data_path = sub_data_path_zepo

In [36]:
data_path

'../../../Data/STC_Z_scored_Epochs/5_sec_time_window//018/'

In [20]:
sfreq = epochs.info["sfreq"]  # the sampling frequency
n_freq_bands = len(Freq_Bands)
min_freq = np.min(list(Freq_Bands.values()))
max_freq = np.max(list(Freq_Bands.values()))

# Prepare the freq points
freqs = np.linspace(min_freq, max_freq, int((max_freq - min_freq) * 4 + 1))

fmin = tuple([list(Freq_Bands.values())[f][0] for f in range(len(Freq_Bands))])
fmax = tuple([list(Freq_Bands.values())[f][1] for f in range(len(Freq_Bands))])

# We specify the connectivity measurements
connectivity_methods = [
    "wpli",
    "dpli",
    "plv",
]
n_con_methods = len(connectivity_methods)

# Compute connectivity over trials
con_epochs = spectral_connectivity_epochs(
    epochs,
    method=connectivity_methods,
    sfreq=sfreq,
    mode="cwt_morlet",
    cwt_freqs=freqs,
    fmin=fmin,
    fmax=fmax,
    faverage=True,
    tmin=tmin,
    tmax=tmax,
    cwt_n_cycles=4,
)

# Plotting
plot_connectivity_circle(con_epochs_array, con_methods, Freq_Bands, roi_names)
plot_global_connectivity(epochs, tmin, tmax, n_connections, con_epochs, Freq_Bands)
plot_connectivity(con_epochs, t_con_max, roi_names, con_methods)
plot_con_matrix(con_data, n_con_methods, connectivity_methods, roi_names, foi)



Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 66 connections
    using t=0.000s..2.000s for estimation (801 points)
    computing connectivity for the bands:
     band 1: 4.0Hz..8.0Hz (17 points)
     band 2: 8.0Hz..13.0Hz (21 points)
     band 3: 13.0Hz..30.0Hz (69 points)
     band 4: 30.0Hz..58.5Hz (115 points)
     band 5: 58.5Hz..61.5Hz (13 points)
     band 6: 61.5Hz..100.0Hz (155 points)
    connectivity scores will be averaged for each band
    using CWT with Morlet wavelets to estimate spectra
    the following metrics will be computed: WPLI, DPLI
    computing cross-spectral density for epoch 1


/tmp/ipykernel_97381/2899385425.py:22: RuntimeWarning: There were no Annotations stored in <EpochsArray |  73 events (all good), 0 – 5 s, baseline off, ~13.4 MB, data loaded,
 '1': 73>, so metadata was not modified.
  con_epochs = spectral_connectivity_epochs(


    computing cross-spectral density for epoch 2
    computing cross-spectral density for epoch 3
    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    computing cross-spectral density for epoch 11
    computing cross-spectral density for epoch 12
    computing cross-spectral density for epoch 13
    computing cross-spectral density for epoch 14
    computing cross-spectral density for epoch 15
    computing cross-spectral density for epoch 16
    computing cross-spectral density for epoch 17
    computing cross-spectral density for epoch 18
    computing cross-spectral density for epoch 19
    computing cross-spectral density for epoch 20
    computing cross-spectral density for epoch 21
    comp